In [1]:
import numpy as np
from math import log
import pandas as pd
import operator
import os

## KNN (유클리드 거리)

In [7]:
directory = 'C:/Users/SAMSUNG/Desktop/new/수행평가/digits/trainingDigits/'
files = os.listdir(directory)

dataset = []
for filename in files:
    file = open(directory + filename)
    dataset.append(file.readlines())
    file.close()

In [8]:
np.array(dataset).shape

(1934, 32)

In [ ]:
# np.array(dataset).reshape(1934,-1).astype('int')

In [9]:
def classify0(inX, dataSet, labels, k): # inX = 1024개(txt파일 1개) 데이터를 입력
    # 나중에 정규화한 데이터를 행별로 inX에 대입함
    # inX에 test를 넣고 dataSet과 labels는 training을 넣음
    # 그 후 결과 확인 부분 print 할떄에만 test_label을 이용함.
    dataSetSize = dataSet.shape[0]
    diffMat = np.tile(inX, (dataSetSize, 1)) - dataSet # inX를 (1934,1)로 만든후 dataSet을 빼줌 -> p-q 거리
    # 행별로 전체 사이즈에 맞춘 후 기존 데이터셋을 뺌 (거리정보 처리과정) -> 이게 거리정보가 됨
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis = 1) # 1934개의 sum값이 나옴
    distances = sqDistances**0.5
    sortedDistIndicies = distances.argsort() # sort했을 떄 작은 값부터 인덱스를 반환 [2,1,3] -> (1,0,2)
    # 1934개의 순위가 출력됨
    classCount = {}
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
        # 해당 label이 있을 경우 그 value에 1을 더함(count하는절차) / 없을 경우 1부터 시작시키라는 코드
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
    # items()는 object를 list로 반환 / iteritems()는 object를 iterator 객체로 반환
    # itemgetter = 0 : key값 기준으로 정렬 / 1 : value값을 기준으로 정렬 <- (key, value) 이렇게 출력됨
    # reverse = True 이므로 큰 값부터 작은 값 순으로 출력
    return sortedClassCount[0][0]

# 여기서 for문을 어떻게 적용할지 고민
# directory = 'C:/Users/SAMSUNG/Desktop/new/수행평가/digits/trainingDigits/'
##files = os.listdir(directory)

def file2matrix(directory):
    files = os.listdir(directory)
    data = []
    for filename in files:
        fr = open(directory + filename)
        data.append(fr.readlines())
        fr.close()
    for i in range(np.array(data).shape[0]):
        for j in range(len(data[i])):
            data[i][j] = list(map(int, list(data[i][j].strip())))
    data = np.array(data)
    returnMat = np.array(data.reshape(len(data),-1))
    # 행 * 3 의 0행렬 생성
    classLabelVector=[]
    for x in range(len(files)):
        label = files[x][:1]
        classLabelVector.append(int(label))
    return returnMat, classLabelVector

def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m,1))
    normDataSet = normDataSet / np.tile(ranges, (m,1)) # element wise divide
    for i in range(normDataSet.shape[0]):
        for j in range(normDataSet.shape[1]):
            if np.isnan(normDataSet[i][j]):
                normDataSet[i][j] = 0
    return normDataSet, ranges, minVals

def datingClassTest():
#    hoRatio = 0.10 # hold out 10%
    datingDataMat_tr, datingLabels_tr = file2matrix('C:/Users/SAMSUNG/Desktop/new/수행평가/digits/trainingDigits/')
    datingDataMat_ts, datingLabels_ts = file2matrix('C:/Users/SAMSUNG/Desktop/new/수행평가/digits/testDigits/')
    normMat_tr, ranges_tr, minVals_tr = autoNorm(datingDataMat_tr)
    normMat_ts, ranges_ts, minVals_ts = autoNorm(datingDataMat_ts)
    # min-max 표준화
    m = normMat_ts.shape[0] # x_train의 행 갯수
    numTestVecs = int(m) # 데이터 obj의 10%를 테스트용으로 사용
    errorCount = 0.0
    for i in range(numTestVecs): # range(test사이즈)
        classifierResult = classify0(normMat_ts[i,:], normMat_tr, datingLabels_tr, 3)
        # classify0(inX, dataSet, labels, k)
        # [numTestVecs:m, :] -> 10% 이후부터 끝까지
        # classity0의 inX에 test_x 대입!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # dataSet에는 training_x / label에는 training_y
        print("the classifier came back with: {}, the real answer is: {}".format(classifierResult, datingLabels_ts[i]))
        # test데이터를 training에 적용한 결과, 실제 test label값 출력
        if (classifierResult != datingLabels_ts[i]): errorCount += 1.0
    print("the total error rate is : {}".format(errorCount/float(numTestVecs))) # error / test횟수
    print("the error count : {}".format(errorCount))

In [10]:
datingClassTest()

C:\Users\SAMSUNG\Desktop\new\anacanda\lib\site-packages\ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in true_divide


the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answe

the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answer is: 1
the classifier came back with: 1, the real answe

the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answer is: 3
the classifier came back with: 3, the real answe

the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answer is: 4
the classifier came back with: 4, the real answe

the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answer is: 6
the classifier came back with: 6, the real answe

the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answer is: 7
the classifier came back with: 7, the real answe

the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answer is: 9
the classifier came back with: 9, the real answe

## Decision Tree

In [2]:
from math import log
import operator

    - array 행, 열 추가
        (np.c_와 np.r_는 list도 가능)
    - 열 추가
        train = np.c_[returnMat, main_labels]
        main_labels.reshape(-1,1)
        C = np.hstack((returnMat, main_labels))
    - 행추가
        C = np.r_[A, [A[2]]]
        C = np.vstack((A,D))
    - list flatten 하게 만들기
        data[i] = sum(data[i], []) # flattened list

In [3]:
def make_data(directory, files):
    data = []
    for filename in files:
        file = open(directory + filename)
        data.append(file.readlines())
        file.close()

# main_labels
    main_labels = []
    for x in range(len(files)):
        label = files[x][:1]
        main_labels.append(label)
    
    for i in range(np.array(data).shape[0]):
        for j in range(len(data[i])):
            data[i][j] = list(map(int, list(data[i][j].strip())))
        
    def flatten(n):
        org = []
        for i in n:
            if (isinstance(i,list)):
                org += flatten(i)
            else:
                org.append(i)
        return org

    for i in range(len(data)):
        data[i] = flatten(data[i])
    return data, main_labels

In [4]:
# 데이터 생성
directory_train = 'C:/Users/Affinity/Desktop/study/1차 수행평가/digits/trainingDigits/'
directory_test = 'C:/Users/Affinity/Desktop/study/1차 수행평가/digits/testDigits/'
files_train = os.listdir(directory_train)
files_test = os.listdir(directory_test)

train, train_labels = make_data(directory_train, files_train)
test, test_labels = make_data(directory_test, files_test)

# labels 생성
labels = list(range(len(train[0])))

In [5]:
# train은 train과 labels를 합쳐야함
for i in range(len(train)):
    train[i].extend([train_labels[i]]) # flattened list

이거 오류나서 맨밑에 del부분 코드 바꾼걸로 돌려야함
바로 밑에꺼는 첫번째 bestFeat label이 지워짐

In [22]:
# 엔트로피 구해보기
# Entropy함수 (적용되는 dataSet마다 before, left, right가 될 수 있음)
def calcShannonEnt(dataSet):
    numEntries = len(dataSet) # dataset의 행 갯수를 numEntries로 지정
    labelCounts = {}
    for featVec in dataSet: # the number of unique elements and their occurance
        currentLabel = featVec[-1] # y값 분리
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
        # 여기까지가 label의 갯수 사전 생성
    shannonEnt = 0.0
    for key in labelCounts: # labelCounts = {'yes' : 2, 'no' : 3}
        prob = float(labelCounts[key]) / numEntries # label별 비율
        shannonEnt -= prob * log(prob, 2) # log base 2
        # shannonEnt는 엔트로피 공식
    return shannonEnt

# 모든 속성을 분리하기
# (자식노드로 분리할 변수 선택시 그 열을 제외한 데이터를 가져오는 함수)
def splitDataSet(dataSet, axis, value): # axis = n번재 행
    retDataSet = []
    for featVec in dataSet: # featVec은 하나의 행
        if featVec[axis] == value: # 각 행의 원소를 모두 돔 / value:는 class로써 classList순환
            reducedFeatVec = featVec[:axis] # chop out(노출하다) axis used for splitting
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet  # 각 행의 axis번째 데이터 빼고 리스트로 출력

# 데이터 분할 시 가장 좋은 속성 선택하기
# Info Gain이 최대로 되는 값 구하는 부분
# 방법 : 1. 각 열별 info gain을 구함 (각 class들의 엔트로피를 구해서 더한 후 baseEntophy에서 빼줌)
#       2. 열별 info gain을 비교해서 가장 큰 info gain을 나타내는 열을 반환한다.
# 데이터 분할 시 가장 좋은 속성 선택하기
# Info Gain이 최대로 되는 값 구하는 부분
# 방법 : 1. 각 열별 info gain을 구함 (각 class들의 엔트로피를 구해서 더한 후 baseEntophy에서 빼줌)
#       2. 열별 info gain을 비교해서 가장 큰 info gain을 나타내는 열을 반환한다.
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1 # the last column is used for the labels
    baseEntropy = calcShannonEnt(dataSet) # Entropy-before
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures): # iterate over all the features(마지막 열인 label 빼고)
        featList = [example[i] for example in dataSet]
        # create a list of all the examples of this feature
        uniqueVals = set(featList) # get a set of unique values
        #  set() 메소드는 자동으로 중복 데이터를 제거하고 unique한 값만 남긴다
        newEntropy = 0.0 # Entropy-after
# Entrophy-after를 구하는 함수 (P(class1), P(class2) 이들의 비율을 곱하므로)
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value) # 각 행의 axis번째 데이터 빼고 array로 출력
            # i번째 열에 value(해당 class)가 있는 행만 추출해서 만든 array
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
            # class별 비율을 곱하는 엔트로피 (P(class1), P(class2) -> Entrophy-before)
        infoGain = baseEntropy - newEntropy # calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain): # compare this to the best gain so far
            bestInfoGain = infoGain # if better than current best, set to best
            bestFeature = i
    return bestFeature # returns an integer
# ---------------------------여기까지가 첫번째 가지치기로 사용할 변수(열) 선택하는 과정----------------------------------

# 재귀적으로 트리 만들기
# 열별 분류할 class 리스트의 갯수를 세는 함수 (ex) 남자, 여자면 2
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
    # itemgetter(1)은 정렬 기준을 value로 하겠다는 뜻
    return sortedClassCount[0][0]
# 해당 열의 class들의 갯수를 세서 가장 높은 비율의 class의 갯수를 반환
# label이 아닌 분류 횟수(변수 갯수)를 정하는 과정
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet] # 행 단위임 / y를 추출하여 classList를 
    # for문을 통해 간단히 array의 요소 추출하기
    # 첫 번째 if는 변수(열) 값이 모두 같아서 분류 기준을 정할 수 없으면 멈춤
    if classList.count(classList[0]) == len(classList): # stop splitting when all of the classes are equal
        return classList[0]                             # 첫 번째 멈춤 조건, 모든 분류 항목이 같을 때 멈춤
    # 두 번째 if는 열이 1개만남았을 떄 가장 많은 class의 갯수를 반환
    if len(dataSet[0]) == 1:                            # stop splitting when there are no more features in dataSet
        return majorityCnt(classList)                   # 두 번째 멈춤 조건, 더 이상 분류할 속성이 없을 때 가장 많은 수를 반환
    # Info Gain이 최대로 되는
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    # 이 밑부터 재귀
    myTree = {bestFeatLabel : {}} # 유일한 값의 리스트를 구함
    del (labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet] # Info Gain이 가장 높은 열
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:] # copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

# 분류 -> dict이 아닐 때까지(가지가 없을 때까지) 내려가서 마지막 값을 반환하는 함수
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree)[0] # inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict):
        classLabel = classify(valueOfFeat, featLabels, testVec)
        # valueOfFeat이 dict인지 부울값으로 반환
        # True면 계속해서 가지따라 내려감
    else:
        classLabel = valueOfFeat
        # dict이 아니면 label값이 나옴 그럼 그 label값을 반환 (yes or no)
    return classLabel

손글씨에서 0만 있는 열 제거시 range를 reversed해서 처음 1이 나올때까지 del로 열 지우기

In [7]:
myTree = createTree(train, labels)

In [34]:
def DecisionTreeTest():
    m = len(test) # x_train의 행 갯수
    numTestVecs = int(m) # 데이터 obj의 10%를 테스트용으로 사용
    errorCount = 0.0
    for i in range(numTestVecs): # range(test사이즈)
        classifierResult = classify(myTree, labels, test[i])
        # classify0(inX, dataSet, labels, k)
        # [numTestVecs:m, :] -> 10% 이후부터 끝까지
        # classity0의 inX에 test_x 대입!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # dataSet에는 training_x / label에는 training_y
        print("the classifier came back with: {}, the real answer is: {}".format(classifierResult, test_labels[i]))
        # test데이터를 training에 적용한 결과, 실제 test label값 출력
        if (classifierResult != test_labels[i]): errorCount += 1.0
    print("the total error rate is : {}".format(errorCount/float(numTestVecs))) # error / test횟수
    print("the error count : {}".format(errorCount))

In [35]:
DecisionTreeTest()

the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 6, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 8, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answer is: 0
the classifier came back with: 0, the real answe

    양쪽에 0만 있는 부분 제거 부분
    - 뒤에서부터랑 앞에서부터를 while문을 통해 찾는다 ( test, train 번갈아 넣어보기 )

In [77]:
test_sub = []
for i in range(len(test[0])):
    test_sub.append([example[i] for example in test])

num = len(test[0]) - 1
while num >= 0:
    if 1 in [example[num] for example in test]:
        print(num)
        break
    else:
        num -= 1

1015


In [76]:
train_sub = []
for i in range(len(test[0])):
        train_sub.append([example[i] for example in test])
        
num = 0
while num < len(train_sub):
    if 1 in train_sub[num]:
        print(num)
        break
    else:
        num +=1

0


In [109]:
# 데이터 생성
directory_train = 'C:/Users/SAMSUNG/Desktop/new/수행평가/digits/trainingDigits/'
directory_test = 'C:/Users/SAMSUNG/Desktop/new/수행평가/digits/testDigits/'
files_train = os.listdir(directory_train)
files_test = os.listdir(directory_test)

train, train_labels = make_data(directory_train, files_train)
test, test_labels = make_data(directory_test, files_test)

In [28]:
for exam in train:
    del exam[:7]
    del exam[-1:]
    
len(train[0])

1016

In [29]:
for exam in test:
    del exam[:7]
    del exam[-1:]
    
len(test[0])

1016

In [30]:
# labels 생성
labels = list(range(len(train[0])))

In [31]:
# train은 train과 labels를 합쳐야함
# 다시 붙여주기
for i in range(len(train)):
    train[i].extend([train_labels[i]]) # flattened list

In [6]:
# 엔트로피 구해보기
# Entropy함수 (적용되는 dataSet마다 before, left, right가 될 수 있음)
def calcShannonEnt(dataSet):
    numEntries = len(dataSet) # dataset의 행 갯수를 numEntries로 지정
    labelCounts = {}
    for featVec in dataSet: # the number of unique elements and their occurance
        currentLabel = featVec[-1] # y값 분리
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
        # 여기까지가 label의 갯수 사전 생성
    shannonEnt = 0.0
    for key in labelCounts: # labelCounts = {'yes' : 2, 'no' : 3}
        prob = float(labelCounts[key]) / numEntries # label별 비율
        shannonEnt -= prob * log(prob, 2) # log base 2
        # shannonEnt는 엔트로피 공식
    return shannonEnt

# 모든 속성을 분리하기
# (자식노드로 분리할 변수 선택시 그 열을 제외한 데이터를 가져오는 함수)
def splitDataSet(dataSet, axis, value): # axis = n번재 행
    retDataSet = []
    for featVec in dataSet: # featVec은 하나의 행
        if featVec[axis] == value: # 각 행의 원소를 모두 돔 / value:는 class로써 classList순환
            reducedFeatVec = featVec[:axis] # chop out(노출하다) axis used for splitting
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet  # 각 행의 axis번째 데이터 빼고 리스트로 출력

# 데이터 분할 시 가장 좋은 속성 선택하기
# Info Gain이 최대로 되는 값 구하는 부분
# 방법 : 1. 각 열별 info gain을 구함 (각 class들의 엔트로피를 구해서 더한 후 baseEntophy에서 빼줌)
#       2. 열별 info gain을 비교해서 가장 큰 info gain을 나타내는 열을 반환한다.
# 데이터 분할 시 가장 좋은 속성 선택하기
# Info Gain이 최대로 되는 값 구하는 부분
# 방법 : 1. 각 열별 info gain을 구함 (각 class들의 엔트로피를 구해서 더한 후 baseEntophy에서 빼줌)
#       2. 열별 info gain을 비교해서 가장 큰 info gain을 나타내는 열을 반환한다.
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1 # the last column is used for the labels
    baseEntropy = calcShannonEnt(dataSet) # Entropy-before
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures): # iterate over all the features(마지막 열인 label 빼고)
        featList = [example[i] for example in dataSet]
        # create a list of all the examples of this feature
        uniqueVals = set(featList) # get a set of unique values
        #  set() 메소드는 자동으로 중복 데이터를 제거하고 unique한 값만 남긴다
        newEntropy = 0.0 # Entropy-after
# Entrophy-after를 구하는 함수 (P(class1), P(class2) 이들의 비율을 곱하므로)
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value) # 각 행의 axis번째 데이터 빼고 array로 출력
            # i번째 열에 value(해당 class)가 있는 행만 추출해서 만든 array
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
            # class별 비율을 곱하는 엔트로피 (P(class1), P(class2) -> Entrophy-before)
        infoGain = baseEntropy - newEntropy # calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain): # compare this to the best gain so far
            bestInfoGain = infoGain # if better than current best, set to best
            bestFeature = i
    return bestFeature # returns an integer
# ---------------------------여기까지가 첫번째 가지치기로 사용할 변수(열) 선택하는 과정----------------------------------

# 재귀적으로 트리 만들기
# 열별 분류할 class 리스트의 갯수를 세는 함수 (ex) 남자, 여자면 2
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key = operator.itemgetter(1), reverse = True)
    # itemgetter(1)은 정렬 기준을 value로 하겠다는 뜻
    return sortedClassCount[0][0]
# 해당 열의 class들의 갯수를 세서 가장 높은 비율의 class의 갯수를 반환
# label이 아닌 분류 횟수(변수 갯수)를 정하는 과정
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet] # 행 단위임 / y를 추출하여 classList를 
    # for문을 통해 간단히 array의 요소 추출하기
    # 첫 번째 if는 변수(열) 값이 모두 같아서 분류 기준을 정할 수 없으면 멈춤
    if classList.count(classList[0]) == len(classList): # stop splitting when all of the classes are equal
        return classList[0]                             # 첫 번째 멈춤 조건, 모든 분류 항목이 같을 때 멈춤
    # 두 번째 if는 열이 1개만남았을 떄 가장 많은 class의 갯수를 반환
    if len(dataSet[0]) == 1:                            # stop splitting when there are no more features in dataSet
        return majorityCnt(classList)                   # 두 번째 멈춤 조건, 더 이상 분류할 속성이 없을 때 가장 많은 수를 반환
    # Info Gain이 최대로 되는
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    # 이 밑부터 재귀
    myTree = {bestFeatLabel : {}} # 유일한 값의 리스트를 구함
    subLabels = labels[:] 
    del (subLabels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet] # Info Gain이 가장 높은 열
    uniqueVals = set(featValues)
    for value in uniqueVals:
        # copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

# 분류 -> dict이 아닐 때까지(가지가 없을 때까지) 내려가서 마지막 값을 반환하는 함수
def classify(inputTree, featLabels, testVec):
    firstStr = list(inputTree)[0] # inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict):
        classLabel = classify(valueOfFeat, featLabels, testVec)
        # valueOfFeat이 dict인지 부울값으로 반환
        # True면 계속해서 가지따라 내려감
    else:
        classLabel = valueOfFeat
        # dict이 아니면 label값이 나옴 그럼 그 label값을 반환 (yes or no)
    return classLabel